In [ ]:
import os, sys
from pathlib import Path
script_dir = Path(os.path.dirname(os.path.abspath('')))
module_dir = str(script_dir)
sys.path.insert(0, module_dir + '/modules')
print(module_dir)

# import the rest of the modules
%matplotlib nbagg
import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt
import arch
import pandas as pd
import tensorflow_probability as tfp
import time  
import sim
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from mpl_toolkits.axes_grid1 import make_axes_locatable

dim = 3
n_particles = int(1e6)
n_subdivisions = 30
save_folder = '../data/Thomas'
n_steps = 500
n_repeats = 10
dt = 0.01
r = 1.0
sigma = np.sqrt(2.)
s = 1.
def mu_np(X, b=0.2):
    x, y, z = np.split(X, 3, axis=-1)
    p = np.sin(s*y) - b*x 
    q = np.sin(s*z) - b*y 
    r = np.sin(s*x) - b*z
    return np.concatenate([p, q, r], axis=-1)

def get_midpoints(x):
    return (x[1:] + x[:-1]) / 2

X0 =  tfp.distributions.MultivariateNormalDiag(scale_diag=tf.ones(dim)).sample(n_particles).numpy()
mc_prob = sim.MCProb(save_folder, n_subdivisions, mu_np, sigma, X0, tick_size=20, title_size=20, cbar_tick_size=15)
mc_prob.ready(n_steps=n_steps, dt=dt, lims=None)
#mc_prob.compute_all(n_steps=n_steps, dt=dt)

C:\Users\pinak\Documents\GitHub\non-grad3D


In [ ]:
def get_net(iter):
    net = arch.LSTMForgetNet(num_nodes=50, num_blocks=3)
    if iter < 5000:
        net.load_weights('../data/Thomas-true-vs-learned/init/Thomas_{}'.format(iter)).expect_partial()
    elif iter < 10000:
        net.load_weights('../data/Thomas-true-vs-learned/Thomas_{}'.format(iter)).expect_partial()
    else:
        net.load_weights('../data/Thomas/Thomas').expect_partial()
    return net

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
div = make_axes_locatable(ax)
cax = div.append_axes('right', '5%', '5%')
grid = mc_prob.get_grid()
low = grid.mins
high = grid.maxs
n_subdivs = 30
x = np.linspace(low[0], high[0], num=n_subdivs+1, endpoint=True).astype('float32')[1:]
y = np.linspace(low[1], high[1], num=n_subdivs+1, endpoint=True).astype('float32')[1:]
z = np.linspace(low[2], high[2], num=n_subdivs+1, endpoint=True).astype('float32')[1:]
x, y, z = np.meshgrid(x, y, z, indexing='ij')
x = x.reshape(-1, 1)
y = y.reshape(-1, 1)
z = z.reshape(-1, 1)
x_ = np.linspace(low[0], high[0], num=n_subdivs, endpoint=True).astype('float32')
y_ = np.linspace(low[1], high[1], num=n_subdivs, endpoint=True).astype('float32')
z_ = np.linspace(low[2], high[2], num=n_subdivs, endpoint=True).astype('float32')
iters = list(range(0, 500, 1)) + list(range(500, 10000, 100)) + [200000]*24
nets = {iter: get_net(iter) for iter in iters}

    
fig_all = plt.figure(figsize=(16, 16))
ax_1l = fig_all.add_subplot(321) 
ax_1m = fig_all.add_subplot(322)


ax_2l = fig_all.add_subplot(323) 
ax_2m = fig_all.add_subplot(324)

ax_3l = fig_all.add_subplot(325) 
ax_3m = fig_all.add_subplot(326)


div_1 = make_axes_locatable(ax_1l)
cax_1 = div_1.append_axes('right', '5%', '5%')
div_2 = make_axes_locatable(ax_2l)
cax_2 = div_2.append_axes('right', '5%', '5%')
div_3 = make_axes_locatable(ax_3l)
cax_3 = div_3.append_axes('right', '5%', '5%')

tick_size = 15
title_size = 20
cbar_tick_size = 20

im = ax_1m.pcolormesh(x_, y_, mc_prob.compute_p2(0, 1, save=False).T, cmap='inferno_r', shading='auto')
cbar = fig_all.colorbar(im, ax=ax_1m)
cbar.ax.tick_params(labelsize=cbar_tick_size)
#cbar.formatter.set_powerlimits((0, 0))
ax_1m.set_aspect("equal")

im = ax_2m.pcolormesh(y_, z_, mc_prob.compute_p2(1, 2, save=False).T, cmap='inferno_r', shading='auto')
cbar = fig_all.colorbar(im, ax=ax_2m)
cbar.ax.tick_params(labelsize=cbar_tick_size)
#cbar.formatter.set_powerlimits((0, 0))
ax_2m.set_aspect("equal")

im = ax_3m.pcolormesh(z_, x_, mc_prob.compute_p2(2, 0, save=False).T, cmap='inferno_r', shading='auto')
cbar = fig_all.colorbar(im, ax=ax_3m)
cbar.ax.tick_params(labelsize=cbar_tick_size)
#cbar.formatter.set_powerlimits((0, 0))
ax_3m.set_aspect("equal")

ax_1m.tick_params(axis='both', which='major', labelsize=tick_size)
ax_1m.tick_params(axis='both', which='minor', labelsize=tick_size)
ax_2m.tick_params(axis='both', which='major', labelsize=tick_size)
ax_2m.tick_params(axis='both', which='minor', labelsize=tick_size)
ax_3m.tick_params(axis='both', which='major', labelsize=tick_size)
ax_3m.tick_params(axis='both', which='minor', labelsize=tick_size)

ax_1m.set_title('Monte Carlo estimate', fontsize=title_size)
ax_1m.set_xlabel('x', fontsize=title_size)
ax_1m.set_ylabel('y', fontsize=title_size)
ax_2m.set_xlabel('y', fontsize=title_size)
ax_2m.set_ylabel('z', fontsize=title_size)
ax_3m.set_xlabel('z', fontsize=title_size)
ax_3m.set_ylabel('x', fontsize=title_size)



def func_cube(frame):
    ax_1l.clear()
    cax_1.cla()
    z_l = np.exp(nets[frame](x, y, z).numpy()).reshape(n_subdivs, n_subdivs, n_subdivs)
    z_1l = tf.reduce_sum(z_l, axis=2).numpy()
    z_2l = tf.reduce_sum(z_l, axis=0).numpy()
    z_3l = tf.reduce_sum(z_l, axis=1).numpy()
    del z_l
    z_1l /= (z_1l.sum() * grid.h[0] * grid.h[1])
    im = ax_1l.pcolormesh(x_, y_, z_1l.T, cmap='inferno_r')
    ax_1l.set_title('Learned solution, iteration = {}'.format(frame), fontsize=title_size)
    ax_1l.tick_params(axis='both', which='major', labelsize=tick_size)
    ax_1l.tick_params(axis='both', which='minor', labelsize=tick_size)
    cbar = fig_all.colorbar(im, cax=cax_1, ax=ax_1l)
    cbar.ax.tick_params(labelsize=cbar_tick_size)
    #cbar.formatter.set_powerlimits((0, 0))
    ax_1l.set_aspect("equal")
    
    ax_2l.clear()
    cax_2.cla() 
    z_2l /= (z_2l.sum() *  grid.h[1] * grid.h[2])
    im = ax_2l.pcolormesh(y_, z_, z_2l.T, cmap='inferno_r')
    ax_2l.tick_params(axis='both', which='major', labelsize=tick_size)
    ax_2l.tick_params(axis='both', which='minor', labelsize=tick_size)
    cbar = fig_all.colorbar(im, cax=cax_2, ax=ax_2l)
    cbar.ax.tick_params(labelsize=cbar_tick_size)
    #cbar.formatter.set_powerlimits((0, 0))
    ax_2l.set_aspect("equal")
    
    ax_3l.clear()
    cax_3.cla()
    z_3l /= (z_3l.sum() *  grid.h[2] * grid.h[0])
    im = ax_3l.pcolormesh(z_, x_, z_3l, cmap='inferno_r')
    ax_3l.tick_params(axis='both', which='major', labelsize=tick_size)
    ax_3l.tick_params(axis='both', which='minor', labelsize=tick_size)
    cbar = fig_all.colorbar(im, cax=cax_3, ax=ax_3l)
    cbar.ax.tick_params(labelsize=cbar_tick_size)
    #cbar.formatter.set_powerlimits((0, 0))
    ax_3l.set_aspect("equal")
    fig_all.subplots_adjust(wspace=0.3, hspace=0.2)
    
    ax_1l.set_ylabel(r'$p_\infty(x, y)$', fontsize=title_size)
    ax_2l.set_ylabel(r'$p_\infty(y, z)$', fontsize=title_size)
    ax_3l.set_ylabel(r'$p_\infty(z, x)$', fontsize=title_size)

        
  
anim = animation.FuncAnimation(fig=fig_all, func=func_cube, frames=iters, repeat=False)
anim.save('thomas-learning.mp4', writer='ffmpeg', fps=24)

In [3]:
print(np.linspace(1, 7, 5, endpoint=True),np.linspace(1, 7, 5, endpoint=False))

[1.  2.5 4.  5.5 7. ] [1.  2.2 3.4 4.6 5.8]
